In [1]:
import numpy as np
from collections import Counter

In [4]:
# Set of points (i,j) that are initially given alongside its value k
def init_graph(puzzle):
    graph = set()
    for i,x in enumerate(puzzle.flatten()):
        if x==-1:
            continue
        div,mod = divmod(i,9)
        graph.add((div,mod,x))
    return graph

# p[i][j]: Probability distribution on the numbers cell ij can take
def init_P(graph):
    P = np.full((9,9,9),1/9)
    for i,j,k in G:
        P[i][j] = [1 if i==k else 0 for i in range(1,10)]
    return P

# Given a 1D collection of numbers, return the number of missing values
def _cost(values):
    return len(set(range(1,10)).difference(set(values)))

# Sum of the number of missing values over each row, column and subblock
def cost(puzzle):
    block_cost = 0
    row_cost = 0
    col_cost = 0

    x,y,z = np.hsplit(puzzle, 3)
    (x1,x2,x3), (y1,y2,y3), (z1,z2,z3) = np.vsplit(x,3), np.vsplit(y,3), np.vsplit(z,3)
    blocks = [x1,x2,x3,y1,y2,y3,z1,z2,z3]
    block_cost = sum([_cost(x.flatten()) for x in blocks])

    for row in puzzle:
        row_cost += _cost(row)
    for col in puzzle.T:
        col_cost += _cost(col)
    return block_cost + row_cost + col_cost

# Return a sample from the probability distribution
def sample(P):
    sample = np.empty((9,9))
    values = range(1,10)
    for i in range(9):
        for j in range(9):
            sample[i][j] = np.random.choice(values,size=None,p=P[i][j])
    return sample

def check_P(P):
    return np.sum(P,axis=2)

In [5]:
# Compute the empirical probabilities (probabilites of each cell taking a value over the samples)
def empirical_probabilities(P, samples):
    p = np.empty((9,9,9))
    N = len(samples)
    for i in range(9):
        for j in range(9):
            numbers_ij = [sample[i][j] for sample in samples]
            count = Counter(numbers_ij)
            p_ij = [count[i]/N for i in range(1,10)]
            p[i][j] = p_ij
    return p

def update_probabilities(alpha,P,empirical_probabilities):
    return alpha * P + (1-alpha) * empirical_probabilities

In [6]:
# Stopping test: If the smallest probability of a cell taking a value is almost 1, a near optimal solution is found
# Beta: Value close to 1
def f(beta,P):
    return np.amin(np.amax(P,axis=2)) > beta

# N: Maximum number of iterations
# Q: Number of samples to take
# Q1: Number of samples used to update probabilities, Q1<=Q
# alpha: Smoothing parameter in range [0,1]
# beta: Value close to 1
def main(graph, N , Q, Q1, alpha, beta):
    P0 = init_P(graph)
    P = P0.copy()
    for _ in range(N):
        if f(beta,P):
            break
        samples = [sample(P) for _ in range(Q)]
        samples.sort(key=cost)
        best_samples = samples[:Q1]
        p = empirical_probabilities(P, best_samples)
        P = update_probabilities(alpha,P, p)
    return P0, P

In [7]:
puzzle = np.loadtxt('puzzle.txt')
G = init_graph(puzzle)

In [8]:
N = 300
Q = 100
Q1 = 10
alpha = 0.8
epsilon = 0.1
beta = 1 - epsilon

P0, P1 = main(G,N,Q,Q1,alpha,beta)
initial_solution = sample(P0)
final_solution = sample(P1)

In [9]:
cost(initial_solution), cost(final_solution)

(62, 0)

In [ ]:
final_solution